In [1]:
# import os
# os.environ['HF_HOME']= "/Users/bytedance/Desktop/void-diffusion"
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy


from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from transformer import Transformer

In [3]:
torch.backends.mps.is_built(),torch.backends.mps.is_available(),torch.cuda.is_available()

(True, True, False)

In [3]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

# device = torch.device('cpu')

memory usage?
flops
reproduce experiments
ablation study
what else except attention/ architecture properties / mechanisms

In [4]:
from datasets import load_dataset
dataset = load_dataset("wmt14", 'de-en')

/Users/bytedance/Desktop/void-diffusion/env/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for wmt14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wmt14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # split + toInt
tokenizer.pad_token = tokenizer.eos_token
tokenizer.vocab_size

50257

In [7]:
src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1
batch_size = 64

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

# # Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [8]:
# for batch in dataset['train'].iter(batch_size):
#     translation = batch['translation']
    
#     source = [sample['de'] for sample in translation]
#     target = [sample['en'] for sample in translation]
#     source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
#     target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
#     source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.long)
#     target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.long)
#     break


In [9]:
def tokenization(batch):
    translation = batch['translation']
    source = [sample['de'] for sample in translation]
    target = [sample['en'] for sample in translation]
    source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
    target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
    source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.long).to(device)
    target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.long).to(device)
    
    return source_tensor, target_tensor

In [10]:
src_data,tgt_data = tokenization(dataset['train'][:1])

In [11]:
for i in enumerate(dataset['train'].iter(1)):
    print(i)
    break

(0, {'translation': [{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}]})


In [12]:
import time
from torch.profiler import profile, record_function, ProfilerActivity

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# total_loss = 0
batch_size = 2
start = time.time()
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    
    for epoch in range(100):
        optimizer.zero_grad()
        
        for i, batch in enumerate(dataset['train'].iter(batch_size)):
            src_data, tgt_data= tokenization(batch)
            output = transformer(src_data, tgt_data[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(transformer.parameters(), 0.5) # ??????
            optimizer.step()
            
            # total_loss += loss.item()
            print(f"Batch: {i+1}, Loss: {loss.item()} Time:{time.time() - start}")
            
            
        print(f"Epoch: {epoch+1}, Loss: {loss.item()} Time:{time.time() - start }")
        break

# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

STAGE:2024-03-29 12:29:36 87132:1571788 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


mps:0 mps:0 mps:0
Batch: 1, Loss: 10.04391860961914 Time:4.158965110778809
mps:0 mps:0 mps:0
Batch: 2, Loss: 3.1517045497894287 Time:7.373347043991089
mps:0 mps:0 mps:0
Batch: 3, Loss: 1.2158067226409912 Time:9.773545980453491
mps:0 mps:0 mps:0
Batch: 4, Loss: 0.47765421867370605 Time:12.793758153915405
mps:0 mps:0 mps:0
Batch: 5, Loss: 0.7807005047798157 Time:15.55637812614441
mps:0 mps:0 mps:0
Batch: 6, Loss: 1.0251295566558838 Time:18.089786052703857
mps:0 mps:0 mps:0
Batch: 7, Loss: 0.40077975392341614 Time:20.57697105407715
mps:0 mps:0 mps:0
Batch: 8, Loss: 0.5027424097061157 Time:23.206475019454956
mps:0 mps:0 mps:0
Batch: 9, Loss: 0.776701033115387 Time:25.774893045425415
mps:0 mps:0 mps:0
Batch: 10, Loss: 0.88686203956604 Time:28.31331205368042
mps:0 mps:0 mps:0
Batch: 11, Loss: 1.1067520380020142 Time:31.506006956100464
mps:0 mps:0 mps:0
Batch: 12, Loss: 1.1559655666351318 Time:34.48108720779419
mps:0 mps:0 mps:0
Batch: 13, Loss: 0.370850533246994 Time:36.56659722328186
mps:0 

In [ ]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")